# 分类

## MNIST
下载数据

In [2]:
from sklearn.datasets import fetch_openml as fetch_mldata

mnist = fetch_mldata('MNIST_784')
print(mnist['DESCR'])

X, y = mnist['data'], mnist['target']
print(X.shape)
print(y.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/datasets/_openml.py:65: RuntimeWarning: Invalid cache, redownloading file
  warn("Invalid cache, redownloading file", RuntimeWarning)


IncompleteRead: IncompleteRead(13367 bytes read)

展示一个数字和它的标签

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

some_digit = X[3600]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap=matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')
plt.show()

print(y[3600])

划分训练集和测试集

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

import numpy as np

shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

## 训练一个二元分类器

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

from sklearn.linear_model import SGDClassifier

# 训练
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

# 测试
sgd_clf.predict([some_digit])


## 性能考核

### 使用交叉验证测量精度

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

In [ ]:
# 蠢笨的分类器
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Nerer5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

因为只有大约10%的图像是数字5，所以如果你猜一张图不是5，90%的机会你都是正确的！

这说明准确率通常无法成为分类器的首要性能指标，特别是当你处理偏斜数据集的时候。

### 混淆矩阵

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)

混淆矩阵能提供大量信息，但有时你可能希望指标更简洁一些。正类预测的准确率是一个有意思的指标，它也称为分类器的精度：
$$
精度 = \frac TP {TP + FP}
$$
TP是真正类的数量，FP是假正类的数量。

精度通常与另一个指标召回率（也称为灵敏度或正类率）一起使用。
$$
召回率 = \frac TP {TP + FN}
$$
FN是假负类的数量。

### 精度和召回率

In [ ]:
from sklearn.metrics import precision_score, recall_score

prediction_score(y_train_5, y_pred)  # == 4344 / (4344 + 1307)

recall_score(y_train_5, y_pred)  # == 4344 / (4344 + 1077)


我们可以很方便的将精度和召回率组合成一个单一的指标，称为$F_1$分数。当你需要一个简单的方法来比较两种分类器时，这是个非常不错的指标。

$F_1$分数是精度和召回率的谐波平均值。正常的平均值平等对待所有的值，而谐波平均值会给予较低的值更高的权重。因此，只有当召回率和精度都很高时，分类器才能得到较高的$F_1$分数。
$$
F_1 = \frac 2 {\frac 1 精度 + \frac 1 召回率} = 2 \times \frac {精度 \times 召回率} {精度 + 召回率} = \frac TP {TP + \frac {FN + FP} 2}
$$

$F_1$分数对具有相近的精度和召回率的分类器更为有利。
遗憾的是，不能同时增加精度和召回率，这称为*精度/召回率权衡*。


In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_pred)


### 精度/召回率权衡

In [ ]:
# 获取分数
y_scores = sgd_clf.decision_function([some_digit])
print(y_scores)

threshold = 0
y_some_digit_pred = (y_scores > threshold)
print(y_some_digit_pred)

# 提高阈值
threshold = 200000
y_some_digit_pred = (y_scores > threshold)
print(y_some_digit_pred)


如何决定使用什么阈值呢？

In [ ]:
from sklearn.metrics import precision_recall_curve

y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method='decision_function')
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

# 绘图
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", labels='Precision')
    plt.plot(thresholds, recalls[:-1], "g-", labels='Recall')
    plt.xlabel('Threshold')
    plt.legend(loc='upper left')
    plt.ylim([0, 1])

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()


还有一种找到好的精度/召回率权衡的方法是直接绘制精度和召回率的函数图

In [ ]:
plt.plot(recalls[:-1], precisions[:-1], "g-")
plt.xlabel('Recall')
plt.ylabel('Precision')

### ROC曲线

还有一种经常和二元分类器一起使用的工具，叫*受试者工作特征曲线*(简称ROC)。它绘制的是真正类率（召回率的别名）和假正类率（FPR）。FPR是被错误分为正类的负类实例比例。

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

plot_roc_curve(fpr, tpr)
plt.show()


有一种比较分类器的方法是测量曲线下面积(AUC)。完美的分类器的ROC AUC等于1，而纯随机分类器的ROC AUC等于0.5。

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

如何决定使用哪种曲线？
有一个经验法则是，当正类非常少见或者更关注假正类而不是假负类时，应该选择PR曲线，反之则时ROC曲线。

训练一个RandomForestClassifier分类器，并比较它和SGDClassifier分类器的ROC曲线和ROC AUC分数。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, mehtod='predict_proba')

Y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, threshold_forest = roc_curve(y_train_5, y_scores_forest)

plt.plot(fpr, tpr, 'b:', label='SGD')
plot_roc_curve(fpr_forest, tpr_forest, 'Random Forest')
plt.legend(loc='bottom right')
plt.show()

# AUC
roc_auc_score(y_train_5, y_scores_forest)

## 多类别分类器